#T-Shirt Detection Attempt

I train a custom object tetector with T-shirts for my final year project. Credits to [Tony607's object detection demo](https://github.com/Tony607/object_detection_demo) which formed the backbone of the implementation.

##Setting Up Chosen Model and Variables

In [0]:
#training steps
train_steps = 1000

#validation steps
test_steps = 50

#name of the object detection model to use
MODEL = 'ssd_mobilenet_v2_coco_2018_03_29'

#name of the pipeline file in tensorflow object detection API
model_config = 'ssd_mobilenet_v2_coco.config'

#how much the model will feed through the network every increment
batch_size = 12

## Clone My Repo to the Colab  Folder

We do this so that we can work with the files needed for training and testing.

In [0]:
import os

#my FYP repo
my_repo = 'https://github.com/frampos/fyp_demo'

#'%' executes certain magic commands (ipython)
#change directory to content folder
#/content is the root folder of Google Colab and has to be appended to all paths used in the notebook
%cd /content

#refer to path of the cloned repo regardless of the location of the repo
cloned_dir = os.path.abspath(os.path.join('.', os.path.basename(my_repo)))

#take changes from remote repo into the current
!git clone {my_repo}
%cd {cloned_dir}
!git pull

print("Cell Done.")

##TensorFlow Object Detection API and its dependencies

I follow the steps for installing the API [here](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/installation.md). The code below installs the API and its dependencies.

In [0]:
%cd /content

#clone the TensorFlow API itself
!git clone https://github.com/tensorflow/models.git

#"protobuf compiler" installation
!apt-get install -qq protobuf-compiler python-pil python-lxml python-tk

#remaining libraries
!pip install Cython
!pip install contextlib2
!pip install jupyter
!pip install matplotlib
!pip install pillow
!pip install pycocotools
!pip install numpy==1.17.5

%cd /content/models/research

#execute protobuf compilation
#the compiler converts .proto files to .py files
!protoc object_detection/protos/*.proto --python_out=.

#add Libraries to PYTHONPATH
#maintain directories of custom Python libraries
import os
os.environ['PYTHONPATH'] += ':/content/models/research/:/content/models/research/slim/'

#verifying installation
!python object_detection/builders/model_builder_test.py

print("Cell Done.")

##Generate TFRecords

API requires TFRecords for training and testing.


In [0]:
%cd {cloned_dir}

#pass arguments to xml_to_csv.py to convert the .xml annotations to one .csv file
!python xml_to_csv.py
!python generate_tfrecord.py --csv_input=data/annotations/train_labels.csv  --image_dir=data/train --output_path=data/annotations/train.record
!python generate_tfrecord.py --csv_input=data/annotations/test_labels.csv  --image_dir=data/test --output_path=data/annotations/test.record

#assign the records and label map to variables
train_record = '/content/fyp_demo/data/annotations/train.record'
test_record = '/content/fyp_demo/data/annotations/test.record'
label_map_pbtxt = '/content/fyp_demo/data/annotations/label_map.pbtxt'

print("Cell Done.")

## Download Base Model

The base model comes from the [TensorFlow model zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md) and comes as a compressed .tar file.

*Disclaimer: This cell was not written by me but was used as part of the implementation. Code was taken from [here](https://gist.github.com/AlaaSenjab/ac61b3b23793f9e07c2aa6ca0cff3285).*


In [0]:
%cd /content/models/research

import os
import shutil
import glob
import urllib.request
import tarfile

#base model comes as a tar file, this concatenates the name defined earlier and appends the tar extension to the string
MODEL_FILE = MODEL + '.tar.gz'

#link to base model downloaded
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'

#save destination
DEST_DIR = '/content/models/research/pretrained_model'

if not (os.path.exists(MODEL_FILE)):
    #model name is downloaded 
    urllib.request.urlretrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)

#must extract the tarfile
tar = tarfile.open(MODEL_FILE)
tar.extractall()
tar.close()

#removes the unextracted tar file once the extraction is complete
os.remove(MODEL_FILE)
if (os.path.exists(DEST_DIR)):
    shutil.rmtree(DEST_DIR)
os.rename(MODEL, DEST_DIR)

print("Cell Done.")

## Changing the Model Configuration

The model config file contains the information of the neural network layers and feature extraction behaviour. There a many configs already available within the TensorFlow object detection API so I decided to use one.


In [0]:
import os

#full path of the model configuration
config_path = os.path.join('/content/models/research/object_detection/samples/configs/', model_config)
assert os.path.isfile(config_path), '`{}` not exist'.format(config_path)

print("Cell Done.")

*Disclaimer: The cell below was not written by me but was used as part of the implementation. Code was taken from [here](https://gist.github.com/Said-Akbar/2d749d497ec6c928037660b6cab3bcb3).*

In [0]:
#regrex library
import re

#set the checkpoint destination folder
fine_tune_checkpoint = os.path.join(DEST_DIR, "model.ckpt")

# the label map generated earlier would count only 1 class as there was only one class label in the .xml, "T-shirt"
num_classes = 1

#read in the .config file
with open(config_path) as f:
    s = f.read()
with open(config_path, 'w') as f:
    
    # fine_tune_checkpoint
    s = re.sub('fine_tune_checkpoint: ".*?"',
               'fine_tune_checkpoint: "{}"'.format(fine_tune_checkpoint), s)
    
    # tfrecord files train and test
    s = re.sub(
        '(input_path: ".*?)(train.record)(.*?")', 'input_path: "{}"'.format(train_record), s)
    s = re.sub(
        '(input_path: ".*?)(val.record)(.*?")', 'input_path: "{}"'.format(test_record), s)

    # label_map_path
    s = re.sub(
        'label_map_path: ".*?"', 'label_map_path: "{}"'.format(label_map_pbtxt), s)

    # Set training batch_size.
    s = re.sub('batch_size: [0-9]+',
               'batch_size: {}'.format(batch_size), s)

    # Set training steps, train_steps
    s = re.sub('train_steps: [0-9]+',
               'train_steps: {}'.format(train_steps), s)
    
    # Set number of classes num_classes
    s = re.sub('num_classes: [0-9]+',
               'num_classes: {}'.format(num_classes), s)
    f.write(s)

print("Cell Done.")

In [0]:
model_dir = 'training/'

# removes any old checkpoints when ran again
!rm -rf {model_dir}
os.makedirs(model_dir, exist_ok=True)

print("Cell Done.")

##Connecting to TensorBoard

Followed a [tutorial](https://www.dlology.com/blog/quick-guide-to-run-tensorboard-in-google-colab/) on how to connect to TensorBoard

In [0]:
#use ngrok to tunnel the connection to your local machine
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
#extract the downloaded zip
!unzip -o ngrok-stable-linux-amd64.zip

In [0]:
#run TensorBoard in the background
LOG_DIR = model_dir
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

print("Cell Done.")

In [0]:
#run ngrok to tunnel TensorBoard port 6006 to the outside world
get_ipython().system_raw('./ngrok http 6006 &')

print("Cell Done.")

In [0]:
#get public URL to access the colab TensorBoard
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

## Training Phase

The API has defined a specific argument to pass to it in order to train the model. The .py file for this can be seen in `/content/models/research/object_detection/model_main.py`

In [0]:
#pass arguemnet to model_man.py to train
!python /content/models/research/object_detection/model_main.py \
    --logtostderr \
    --pipeline_config_path={config_path} \
    --model_dir={model_dir} \
    --num_train_steps={train_steps} \
    --num_eval_steps={test_steps} \
    
print("Cell Done.")

## Export Post-Training Results
Save the frozen inference graph for the object detection in the upcoming stages.

Disclaimer: Code in the cell below was not written by me but was used as part of the system.

In [0]:
import re
import numpy as np

#directory to save the frozen graph
output_dir = './post_training_model'

#finds the latest model checkpoint using regrex
lst = os.listdir(model_dir)
lst = [l for l in lst if 'model.ckpt-' in l and '.meta' in l]
steps=np.array([int(re.findall('\d+', l)[0]) for l in lst])
last_model = lst[steps.argmax()].replace('.meta', '')

last_model_path = os.path.join(model_dir, last_model)

#run export_inference_graph.py in the API using argument
!python /content/models/research/object_detection/export_inference_graph.py \
    --input_type=image_tensor \
    --pipeline_config_path={config_path} \
    --output_directory={output_dir} \
    --trained_checkpoint_prefix={last_model_path}
    
print("Cell Done.")

## Download the Frozen Inference File

In [0]:
import os

#full path for the inference graph
frozen_inference = os.path.join(os.path.abspath(output_dir), "frozen_inference_graph.pb")
assert os.path.isfile(frozen_inference), '`{}` not exist'.format(frozen_inference)

print("Cell Done.")

#Object Detection and Visualisation

Disclaimer: This is tutorial code within the API from `models/object_detection/object_detection_tutorial.ipynb` and was used as part of the implementation.


In [0]:
import os
import glob

#resize the images and put them in he new folder
!python resize.py

#path to the model checkpoint used for detection
PATH_TO_CKPT = frozen_inference

#path to labels
PATH_TO_LABELS = label_map_pbtxt

#path of the images for the inference test
PATH_TO_TEST_IMAGES_DIR =  os.path.join(cloned_dir, "test")

assert os.path.isfile(frozen_inference)
assert os.path.isfile(PATH_TO_LABELS)
TEST_IMAGE_PATHS = glob.glob(os.path.join(PATH_TO_TEST_IMAGES_DIR, "*.*"))

print("Cell Done.")

In [0]:
%cd /content/models/research/object_detection

import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")

from object_detection.utils import ops as utils_ops

# This is needed to display the images.
%matplotlib inline

from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')


label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(
    label_map, max_num_classes=num_classes, use_display_name=True)
category_index = label_map_util.create_category_index(categories)


def load_image_into_numpy_array(image):
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape(
        (im_height, im_width, 3)).astype(np.uint8)

# Size, in inches, of the output images.
IMAGE_SIZE = (12, 8)


def run_inference_for_single_image(image, graph):
    with graph.as_default():
        with tf.Session() as sess:
            # Get handles to input and output tensors
            ops = tf.get_default_graph().get_operations()
            all_tensor_names = {
                output.name for op in ops for output in op.outputs}
            tensor_dict = {}
            for key in [
                'num_detections', 'detection_boxes', 'detection_scores',
                'detection_classes', 'detection_masks'
            ]:
                tensor_name = key + ':0'
                if tensor_name in all_tensor_names:
                    tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
                        tensor_name)
            if 'detection_masks' in tensor_dict:
                # The following processing is only for single image
                detection_boxes = tf.squeeze(
                    tensor_dict['detection_boxes'], [0])
                detection_masks = tf.squeeze(
                    tensor_dict['detection_masks'], [0])
                # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
                real_num_detection = tf.cast(
                    tensor_dict['num_detections'][0], tf.int32)
                detection_boxes = tf.slice(detection_boxes, [0, 0], [
                                           real_num_detection, -1])
                detection_masks = tf.slice(detection_masks, [0, 0, 0], [
                                           real_num_detection, -1, -1])
                detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
                    detection_masks, detection_boxes, image.shape[0], image.shape[1])
                detection_masks_reframed = tf.cast(
                    tf.greater(detection_masks_reframed, 0.5), tf.uint8)
                # Follow the convention by adding back the batch dimension
                tensor_dict['detection_masks'] = tf.expand_dims(
                    detection_masks_reframed, 0)
            image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

            # Run inference
            output_dict = sess.run(tensor_dict,
                                   feed_dict={image_tensor: np.expand_dims(image, 0)})

            # all outputs are float32 numpy arrays, so convert types as appropriate
            output_dict['num_detections'] = int(
                output_dict['num_detections'][0])
            output_dict['detection_classes'] = output_dict[
                'detection_classes'][0].astype(np.uint8)
            output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
            output_dict['detection_scores'] = output_dict['detection_scores'][0]
            if 'detection_masks' in output_dict:
                output_dict['detection_masks'] = output_dict['detection_masks'][0]
    return output_dict


#Visualising the detection boxes and plotting the test images
for image_path in TEST_IMAGE_PATHS:
    image = Image.open(image_path)
    # the array based representation of the image will be used later in order to prepare the
    # result image with boxes and labels on it.
    image_np = load_image_into_numpy_array(image)
    # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
    image_np_expanded = np.expand_dims(image_np, axis=0)
    # Actual detection.
    output_dict = run_inference_for_single_image(image_np, detection_graph)
    # Visualization of the results of a detection.
    vis_util.visualize_boxes_and_labels_on_image_array(
        image_np,
        output_dict['detection_boxes'],
        output_dict['detection_classes'],
        output_dict['detection_scores'],
        category_index,
        instance_masks=output_dict.get('detection_masks'),
        use_normalized_coordinates=True,
        line_thickness=8)
    plt.figure(figsize=IMAGE_SIZE)
    plt.imshow(image_np)